# Table of Contents

>**1. Data and Libraries Import**  
**2. Security Implications**  
**3. Customer Behavior across Regions**  
**4. Exclusion Flag**  
**5. Customer Profiling**  
>>5.1 By Age    
         5.2 By Income      
         5.3 By Purchases      
5.4 By Number of Dependants  

>**6. Visualizations**  
>>6.1. Busiest days of the week  
>>6.2. Busiest hours of the day  
>>6.3. Expenditure vs Time of the Day  
>>6.4. Departments with the highest frequency of product orders  
>>6.5. Distribution among users in regards to their brand loyalty  
>>6.6. Age-wise Brand Loyalty  
>>6.7. Household status vs Brand Loyalty  
>>6.8. Income vs Brand Loyalty  
>>6.9. Brand Loyalty by Region  
>>6.10. Department Preferences of Loyal customers  
>>6.11. Age-group wise department preferences  
>>6.12. Income vs Dept Preference  
>>6.13. Household Status vs Department Preference  
>>6.14. Region wise Department Preferences  
>>6.15. Age-group vs Spending flag  
>>6.16. Region wise spending trend  
>>6.17. Age wise most active hours  
>>6.18. Region wise most active hours  
>>6.19. Household Status wise most active hours  
>>6.20. Age wise most active days  
>>6.21. Region wise most active days  
>>6.22. Household Status wise most active days  

>**7. Aggregation**  
>>7.1. Department Re-order Frequency Aggregation    
>>7.2. Age wise usage Aggregation  
>>7.3. Age wise purchase Aggregation  
>>7.4. Income wise Usage Frequency  
>>7.5. Household Status Usage Frequency  
>>7.6. Household Status Spending Aggregation  
>>7.7. Departments and Price tags  



## 1. Data and Libraries Import

**Question 1. Import the data set you exported in Exercise 4.9 into a new Jupyter notebook. This should be the merged version of your data that contains the new customers column.**

In [ ]:
# Import the required libraries
import pandas as pd
from pandas.plotting import andrews_curves
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import plotly.graph_objects as go
import datashader as ds

In [ ]:
# Create a Path variable 
path=r'C:\Users\sands\Documents\01-2022 Instacart Basket Analysis'

In [ ]:
# Import the orders, products and customers merged dataset
df_merged=pd.read_pickle(os.path.join(path,'02 Data', 'Prepared Data','Instacart_data0130.pkl'))

In [ ]:
df_merged.head(10)


## 2. Security Implications

**Question 2. Consider any security implications that might exist for this new data. You’ll need to address any PII data in the data before continuing your analysis.**

In [ ]:
# Fetch the column info
df_merged.info()

In [ ]:
# View the first few rows and look for security implications
df_merged.head(10)

**Note: The customer information in the dataframe are PII data but it is fabricated for project case. If it were actual data, it could be handled in the following ways:**

**1. Encrypt the names columns**

**2. Create a new subset without the names and use it for the purpose of analysis**

**3. Drop the first and last name columns altogether**




## 3. Customer Behavior across Regions

**Question 3. The Instacart officers are interested in comparing customer behavior in different geographic areas. Create a regional segmentation of the data. You’ll need to create a “Region” column based on the “State” column from your customers data set.**

**o	Use the region information in this Wikipedia article to create your column (you only need to create regions, not divisions).**

**o	Determine whether there’s a difference in spending habits between the different U.S. regions. (Hint: You can do this by crossing the variable you just created with the spending flag.)**

In [ ]:
# Check the State column frequency
df_merged['State'].value_counts(dropna=False)

In [ ]:
# Assign the regions to seperate lists
region1_northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 
                     'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
region2_midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 
                   'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']
region3_south = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia',
                 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 
                 'Tennessee', 'Mississippi', 'Alabama','Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
region4_west = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska',
                'Washington', 'Oregon', 'California', 'Hawaii']

In [ ]:
# Assign a region to each Customer based on their States
df_merged.loc[df_merged['State'].isin(region1_northeast), 'Region'] = 'NorthEast'
df_merged.loc[df_merged['State'].isin(region2_midwest), 'Region'] = 'MidWest'
df_merged.loc[df_merged['State'].isin(region3_south), 'Region'] = 'South'
df_merged.loc[df_merged['State'].isin(region4_west), 'Region'] = 'West'

In [ ]:
# Check the Frequency of newly assigned Region column
df_merged['Region'].value_counts(dropna=False)

In [ ]:
# Check the shape of the dataframe
df_merged.shape

In [ ]:
# Create a crosstab to check if there is a correlation between spending habits and region
crosstab = pd.crosstab(df_merged['Region'], df_merged['spending_flag'], dropna = False)

In [ ]:
crosstab

## 4. Exclusion Flag

**Question 4. The Instacart CFO isn’t interested in customers who don’t generate much revenue for the app. Create an exclusion flag for low-activity customers (customers with less than 5 orders) and exclude them from the data. Make sure you export this sample.**

In [ ]:
# Create max_order column with aggregated oder_number by customer_id 
df_merged['max_order'] = df_merged.groupby(['user_id'])['order_number'].transform(np.max)

In [ ]:
# Check the output
df_merged.head(10)

In [ ]:
# Assign the customer an elimination flag if they have less than 5 orders
df_merged.loc[df_merged['max_order'] < 5, 'elimination_flag'] = 'Low Activity'
df_merged.loc[df_merged['max_order'] >= 5, 'elimination_flag'] = 'High Activity'

In [ ]:
# Check the frequency of high and low activity customers
df_merged['elimination_flag'].value_counts()

In [ ]:
# Delete the max_order column
df_1 = df_merged.drop(columns = ['max_order'])

In [ ]:
# Check the output
df_1.info()

In [ ]:
# Export this dataset
df_1.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'Instacart_data0130.pkl'))

## 5. Customer Profiling

**Question 5. The marketing and business strategy units at Instacart want to create more-relevant marketing strategies for different products and are, thus, curious about customer profiling in their database. Create a profiling variable based on age, income, certain goods in the “department_id” column, and number of dependents. You might also use the “orders_day_of_the_week” and “order_hour_of_day” columns if you can think of a way they would impact customer profiles. (Hint: As an example, try thinking of what characteristics would lead you to the profile “Single adult” or “Young parent.”)**

## - By Age:

In [ ]:
# Check the Stats of Age Column
df_1['Age'].describe()

**The stats show:**  
Mean age : 49  
Minimum age : 18  
Maximum age : 81

**Thus the age groups can be divided into following categories:**  
18-29 - Emerging Adult  
30-39 - Young Adult  
40-59 - Middle-Aged Adult  
60-81 - Older Adult  

In [ ]:
# Assign an age group to each customer based on this information
df_1.loc[(df_1['Age'] >=18) & (df_1['Age'] <= 29), 'age_group'] = 'Emerging Adult'
df_1.loc[(df_1['Age'] >=30) & (df_1['Age'] <= 39), 'age_group'] = 'Young Adult'
df_1.loc[(df_1['Age'] >=40) & (df_1['Age'] <= 59), 'age_group'] = 'Middle-aged Adult'
df_1.loc[df_1['Age'] >=60, 'age_group'] = 'Older Adult'

In [ ]:
# Check the newly assigned age group column frequency
df_1['age_group'].value_counts(dropna=False)

## - By Income

In [ ]:
# Check the Stats of Income Column
df_1['income'].describe()

**The stats show:**  
Mean Income: 99,437  
Maximum Income: 593,901  
Minimum Income: 25,903  
75th percentile: 127,912  

**Based on this the customers can be divided into following categories:**  
25,000-50,000- Low   
50,000-100,000- Middle   
100,000-300,000- Upper Middle   
300,000 above- High  

In [ ]:
# Based on the above classification, assign an income group to each customer
df_1.loc[(df_1['income'] >= 25000) & (df_1['income'] <= 50000), 'Income_Category'] = 'Low'
df_1.loc[(df_1['income'] > 50000) & (df_1['income'] <= 100000), 'Income_Category'] = 'Middle'
df_1.loc[(df_1['income'] > 100000) & (df_1['income'] <= 300000), 'Income_Category'] = 'Upper Middle'
df_1.loc[df_1['income'] > 300000, 'Income_Category'] = 'High'

In [ ]:
# Check the frequency of the new column
df_1['Income_Category'].value_counts(dropna=False)

## - By Purchases

In [ ]:
# Import the wrangled departments dataset
df_dep=pd.read_csv(os.path.join(path,'02 Data','Prepared Data','Departments.csv'))

In [ ]:
# Check the imported dataframe
df_dep.head(21)

In [ ]:
# Convert the Departments dataframe to a data dictionary
data_dict = df_dep.to_dict('index')

In [ ]:
# View the data dictionary
data_dict

In [ ]:
# Map the deparment_id column in the merged full dataframe with the name in Departments data dictionary
df_1['department'] = df_1['department_id'].map(data_dict)

In [ ]:
# Check the frequency of the new column added for the department wise analysis
df_1C.value_counts(dropna=False)

**This data can be used to furthur derive the following insights:**  
- Based on weather a customer has purchases in the babies department they can be classified as Young Parent
- If a customer has purchases in the pet department they can be classified as Pet Parent
- If a customer has no purchases only in the deli,breakfast,produce,department, they are likely Vegetarian

## - By Number of Dependants

In [ ]:
# Check the Stats of Income Column
df_1['number_of_dependants'].describe()

**The stats show:**  
Mean no of dependants: 2  
Maximum no of dependants:3  
Minimum no of dependants:0 


In [ ]:
# Assign group based on purchases and number of dependants
df_1.loc[df_1['number_of_dependants'] == 0, 'Household_Status'] = 'Single'
df_1.loc[(df_1['department_id'] == 17) & (df_1['number_of_dependants'] >= 1), 'Household_Status'] = 'New Parent'
df_1.loc[(df_1['department_id'] != 17) & (df_1['number_of_dependants'] >= 1), 'Household_Status'] = 'Older Parent'

In [ ]:
# Check the frequency of the household_status column
df_1['Household_Status'].value_counts(dropna=False)

In [ ]:
# Assign Flag to users who have ordered Pet supplies
df_1.loc[df_1['department_id'] == 7, 'Pet Parent'] = 'Yes'
df_1.loc[df_1['department_id'] != 7, 'Pet Parent'] = 'No'

In [ ]:
# Check the frequency of the Pet Parent column
df_1['Pet Parent'].value_counts(dropna=False)

## 6. Visualizations

**1. Busiest days of the week**

In [ ]:
#Draw a plot bar chart
df_1['orders_day_of_week'].value_counts().plot(kind='bar', color=[ 'olive', 'lightseagreen', 'gold',
                                                                                    'coral', 'seagreen', 'goldenrod', 'yellowgreen'] 
                                               ,figsize=(6,6),alpha=0.7, edgecolor='black')
# Set the labels and plot title
plt.title('Bar Plot of Busiest Days of the Week')
plt.xlabel('Days of the Week')
plt.ylabel('Frequency of Orders')
plt.grid(True)

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations','Task 4.10', 'busiest_day_of_week.png'))

**2. Busiest hours of the day**

In [ ]:
# Set the figure size
plt.rcParams["figure.figsize"] = [6, 6]

x = df_1['order_hour_of_day']
num_bins = 12

# the histogram of the data
n, bins, patches = plt.hist(x, num_bins, facecolor='olive', alpha=0.7, edgecolor='black')

plt.xlabel('Hour of the Day')
plt.ylabel('Order Frequency')
plt.title('Histogram showing busiest hours of the day')
plt.xlim(0, 24)
plt.grid(True)

# Save the histogram
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations','Task 4.10', 'busiest_hours_of_day.png'))


# Display the histogram
plt.show()

**3. Expenditure vs Time of the Day**

In [ ]:
# Create a 70/30 split data
np.random.seed(5)

In [ ]:
# Assign the random values to a list dev
dev = np.random.rand(len(df_1)) <= 0.7

In [ ]:
dev

In [ ]:
# Divide the dataset into two categories
big = df_1[dev]
small = df_1[~dev]

In [ ]:
# check the length of the df dataframe
len(df_1)

In [ ]:
# Check the sum of the length of the two subsets
len(big) + len(small)

In [ ]:
# Create a subset with only the required columns for line chart
df_2 = small[['order_hour_of_day','prices']]

In [ ]:
# Draw a line chart
line_chart_order_hour_of_day_prices = sns.lineplot(data = df_2, x = 'order_hour_of_day',y = 'prices',
                                                   markers=True, palette='#2ca02c', 
                                                   linewidth=2.5).set(title='Hour of the Day vs. Prices')

plt.xlabel('Hour of the Day')
plt.ylabel('Price')
plt.grid(True)

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations','Task 4.10', 'expenditure_vs_time.png'))

**4. Departments with the highest frequency of product orders**

In [ ]:
#Draw a plot bar chart
df_1['department'].value_counts().plot(kind='bar', color=['olive', 'lightseagreen', 'gold', 'coral', 'seagreen',
                                                          'goldenrod', 'yellowgreen'],figsize=(8,8),alpha=1, edgecolor='black')
# Set the labels and plot title
plt.title('Bar Plot of Departments')
plt.xlabel('Department')
plt.ylabel('Frequency of Orders')
plt.grid(True)

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations','Task 4.10', 'departments_and_order_frequency.png'))

**5. Distribution among users in regards to their brand loyalty**

In [ ]:
# Draw a plot bar chart
df_1['loyalty_flag'].value_counts().plot(kind='barh', color=['darkgoldenrod','goldenrod','khaki'],
                                         figsize=(13,4),edgecolor='black')

# Set the labels and plot title
plt.title('Bar Plot of Customer Count by Loyalty Levels')
plt.ylabel('Loyalty Level')
plt.xlabel('Frequency of Customer')

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'customer_loyalty.png'))

**6. Age-wise Brand Loyalty**

In [ ]:
# Check if there is a correlation between age groups and brand loyalty
brand_loyalty_by_age = pd.crosstab(df_1['loyalty_flag'], df_1['age_group'])

In [ ]:
brand_loyalty_by_age

In [ ]:
# Create a stacked bar chart
stacked_age_loyalty_bar = brand_loyalty_by_age.plot(kind = 'bar', stacked = True, 
                                                    color = ['darkorange', 'gold', 'darkgoldenrod','goldenrod','khaki'],figsize=(6,6))

# Set the labels and plot title
plt.title('Bar Plot of Age vs Brand Loyalty')
plt.xlabel('Loyalty Level')
plt.ylabel('Count of Customers')

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'customer_age_vs_loyalty.png'))


**7. Household status vs Brand Loyalty**

In [ ]:
# Check if there is a correlation between household status and brand loyalty
lylty_by_household_status = pd.crosstab(df_1['Household_Status'], df_1['loyalty_flag'])

In [ ]:
lylty_by_household_status

In [ ]:
# Create a bar chart
bar_lylty_by_household_status = lylty_by_household_status.plot(kind = 'bar', color = ['khaki','goldenrod','darkgoldenrod'],figsize=(6,6))

# Set the labels and plot title
plt.title('Bar Plot of Household Status vs Brand Loyalty')
plt.xlabel('Loyalty Level')
plt.ylabel('Count of Customers')

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'lylty_household_status.png'))


**8. Income vs Brand Loyalty**

In [ ]:
# Check if there is a correlation between income groups and brand loyalty
brand_loyalty_by_income = pd.crosstab(df_1['loyalty_flag'], df_1['Income_Category'])

In [ ]:
brand_loyalty_by_income

In [ ]:
# Create a stacked bar chart
bar_lylty_by_income = brand_loyalty_by_income.plot(kind = 'bar', color = ['khaki','goldenrod','darkgoldenrod','orange'],figsize=(6,6))

# Set the labels and plot title
plt.title('Bar Plot of Income Category vs Brand Loyalty')
plt.xlabel('Loyalty Level')
plt.ylabel('Count of Customers')

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'income_ctgry_vs_lylty.png'))


**9. Brand Loyalty by Region**

In [ ]:
# Check if some regions have more loyal customers than others
loyalty_by_region = pd.crosstab(df_1['loyalty_flag'], df_1['Region'])

In [ ]:
loyalty_by_region.to_clipboard()

In [ ]:
# Create a stacked bar chart
bar_loyalty_by_region = loyalty_by_region.plot(kind = 'barh', stacked=True, color = ['darkorange', 'gold', 'darkgoldenrod',
                                                                                    'goldenrod'],figsize=(6,6))

# Set the labels and plot title
plt.title('Brand Loyalty by Region')
plt.ylabel('Loyalty Level')
plt.xlabel('Count of Customers')

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'loyalty_vs_region.png'))


**10. Department Preferences of Loyal customers**

In [ ]:
# Check if there is a correlation between departments and brand loyalty
brand_loyalty_department = pd.crosstab(df_1['department_id'],df_1['loyalty_flag'])

In [ ]:
brand_loyalty_department

In [ ]:
# Create a stacked bar chart
bar_brand_loyalty_department = brand_loyalty_department.plot(kind = 'bar', color = ['darkorange', 'gold', 'darkgoldenrod',
                                                                                    'goldenrod'],figsize=(8,8))

# Set the labels and plot title
plt.title('Brand Loyalty vs Department Preferences')
plt.ylabel('Frequency')
plt.xlabel('Department')

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'lylty_vs_dept.png'))


**11. Age-group wise department preferences**

In [ ]:
# Check which departments are most popular among different age groups
age_department_preference = pd.crosstab(df_1['department_id'],df_1['age_group'])

In [ ]:
age_department_preference

In [ ]:
plt.rcParams["figure.autolayout"] = True

# Create a stacked area chart
bar_age_dpt_preference = age_department_preference.plot(kind = 'area',color = ['olivedrab', 'yellowgreen', 'greenyellow',
                                                                                    'yellow'],figsize=(8,8))

# Set the labels and plot title
plt.title('Age vs Department Preferences')
plt.ylabel('Frequency')
plt.xlabel('Department')
plt.grid(True)
plt.xlim(1, 22)
plt.xticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22])

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'age_vs_dept.png'))


**12. Income vs Dept Preference**

In [ ]:
# Check which departments are most popular among different income groups
income_dpt_preference = pd.crosstab(df_1['department_id'],df_1['Income_Category'])

In [ ]:
income_dpt_preference

In [ ]:
plt.rcParams["figure.autolayout"] = True

# Create a stacked area chart
income_dpt_preference = income_dpt_preference.plot(kind = 'area',color = ['olivedrab', 'yellowgreen', 'greenyellow',
                                                                                    'yellow'],figsize=(8,8))

# Set the labels and plot title
plt.title('Income vs Department Preferences')
plt.ylabel('Frequency')
plt.xlabel('Department')
plt.grid(True)
plt.xlim(1, 22)
plt.xticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22])

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'income_vs_dept.png'))


**13. Household Status vs Department Preference**

In [ ]:
# Check which departments are most popular among different household sizes
hhld_dpt_preference = pd.crosstab(df_1['department_id'],df_1['Household_Status'])

In [ ]:
hhld_dpt_preference

In [ ]:
plt.rcParams["figure.autolayout"] = True

# Create a stacked area chart
plt_hhld_dpt_preference = hhld_dpt_preference.plot(kind = 'area',color = ['olivedrab', 'yellowgreen', 'greenyellow',
                                                                                    'yellow'],figsize=(8,8))

# Set the labels and plot title
plt.title('Household Size vs Department Preferences')
plt.ylabel('Frequency')
plt.xlabel('Department')
plt.grid(True)
plt.xlim(1, 22)
plt.xticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22])

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'household_vs_dept.png'))


**14. Region wise Department Preferences**

In [ ]:
# Check which departments are most popular in different regions
region_dpt_preference = pd.crosstab(df_1['department_id'],df_1['Region'])

In [ ]:
region_dpt_preference

In [ ]:
plt.rcParams["figure.autolayout"] = True

# Create a stacked area chart
region_dpt_preference = region_dpt_preference.plot(kind = 'area',color = ['olivedrab', 'yellowgreen', 'greenyellow',
                                                                                    'yellow'],figsize=(8,8))

# Set the labels and plot title
plt.title('Region wise Department Preferences')
plt.ylabel('Frequency')
plt.xlabel('Department')
plt.grid(True)
plt.xlim(1, 22)
plt.xticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22])

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'regionwise_dept.png'))


**15. Age-group vs Spending flag**

In [ ]:
# Check which age groups have most number of High Spenders
age_vs_spending = pd.crosstab(df_1['spending_flag'],df_1['age_group'])

In [ ]:
age_vs_spending

In [ ]:
# Create a bar chart
age_vs_spending = age_vs_spending.plot(kind = 'bar', color = ['seagreen', 'mediumseagreen', 'green',
                                                                                    'mediumspringgreen'],figsize=(8,8))

# Set the labels and plot title
plt.title('Age vs Spending Trends')
plt.ylabel('Age Group')
plt.xlabel('Spending Trend')

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'age_spending_habit.png'))


**16. Region wise spending trend**

In [ ]:
# Check which regions have most number of High Spenders
region_vs_spending = pd.crosstab(df_1['spending_flag'],df_1['Region'])

In [ ]:
region_vs_spending 

In [ ]:
# Create a bar chart
region_vs_spending  = region_vs_spending.plot(kind = 'bar', color = ['seagreen', 'mediumseagreen', 'green',
                                                                                    'mediumspringgreen'],figsize=(8,8))

# Set the labels and plot title
plt.title('Region wise Spending Trends')
plt.ylabel('Region')
plt.xlabel('Spending Trend')

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'region_spending_habit.png'))


**17. Age wise most active hours**

In [ ]:
# Create a crosstab between age group and hour of the day
active_hours_vs_age = pd.crosstab(df_1['order_hour_of_day'],df_1['age_group'])

In [ ]:
active_hours_vs_age

In [ ]:
# Create a bar chart
bar_active_hours_vs_age  = active_hours_vs_age.plot(kind = 'bar', color = ['olive', 'lightseagreen', 'gold',
                                                                                    'coral'],figsize=(8,8))

# Set the labels and plot title
plt.title('Age wise Active Hours')
plt.xlabel('Hour of the Day')
plt.ylabel('Frequency')

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'age_wise_active_hours.png'))


**18. Region wise most active hours**

In [ ]:
# Create a crosstab between region and hour of the day
region_vs_hours = pd.crosstab(df_1['order_hour_of_day'],df_1['Region'])

In [ ]:
region_vs_hours

In [ ]:
# Create a bar chart
bar_region_vs_hours  = region_vs_hours.plot(kind = 'bar', color = ['olive', 'lightseagreen', 'gold',
                                                                                    'coral'],figsize=(8,8))

# Set the labels and plot title
plt.title('Region wise Active Hours')
plt.xlabel('Hour of the Day')
plt.ylabel('Frequency')

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'region_wise_active_hours.png'))


**19. Household Status wise most active hours**

In [ ]:
# Create a crosstab between household status and hour of the day
hhld_status_vs_hours = pd.crosstab(df_1['order_hour_of_day'],df_1['Household_Status'])

In [ ]:
hhld_status_vs_hours

In [ ]:
# Create a bar chart
bar_hhld_status_vs_hours  = hhld_status_vs_hours.plot(kind = 'bar', color = ['olive', 'lightseagreen', 'gold',
                                                                                    'coral'],figsize=(8,8))

# Set the labels and plot title
plt.title('Household Status wise Active Hours')
plt.xlabel('Hour of the Day')
plt.ylabel('Frequency')

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'hhld_status_vs_hours.png'))


**20. Age wise most active days**

In [ ]:
# Create a crosstab between age and day of the week
age_vs_active_days = pd.crosstab(df_1['orders_day_of_week'],df_1['age_group'])

In [ ]:
age_vs_active_days

In [ ]:
# Create a bar chart
bar_age_vs_active_days  = age_vs_active_days.plot(kind = 'barh', color = ['green', 'skyblue', 'yellow',
                                                                                    'orange'],figsize=(8,8))

# Set the labels and plot title
plt.title('Age wise Active Days')
plt.ylabel('Day of the Week')
plt.xlabel('Frequency')

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'age_vs_active_days.png'))


**21. Region wise most active days**

In [ ]:
# Create a crosstab between region and day of the week
region_vs_active_days = pd.crosstab(df_1['orders_day_of_week'],df_1['Region'])

In [ ]:
region_vs_active_days

In [ ]:
# Create a bar chart
bar_region_vs_active_days = region_vs_active_days.plot(kind = 'barh', color = ['green', 'skyblue', 'yellow',
                                                                                    'orange'],figsize=(8,8))

# Set the labels and plot title
plt.title('Region wise Active Days')
plt.ylabel('Day of the Week')
plt.xlabel('Frequency')

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'region_vs_active_days.png'))


**22. Household Status wise most active days**

In [ ]:
# Create a crosstab between household status and day of the week
hhld_status_vs_active_days = pd.crosstab(df_1['orders_day_of_week'],df_1['Household_Status'])

In [ ]:
hhld_status_vs_active_days

In [ ]:
# Create a bar chart
bar_hhld_status_vs_active_days = hhld_status_vs_active_days.plot(kind = 'barh', color = ['green', 'skyblue', 'yellow',
                                                                                    'orange'],figsize=(8,8))

# Set the labels and plot title
plt.title('Household Status wise Active Days')
plt.ylabel('Day of the Week')
plt.xlabel('Frequency')

# Save the plot
plt.savefig(os.path.join(path,'04 Analysis', 'Visualizations', 'Task 4.10', 'hhld_status_vs_active_days.png'))


## 7. Aggregation

**Question 7. Aggregate the max, mean, and min variables on a customer-profile level for usage frequency and expenditure**

**1. Department Re-order Frequency Aggregation**

In [ ]:
dept_order_agg = df_1.groupby('department_id').agg({'reordered':['mean', 'min','max']})

In [ ]:
dept_order_agg.to_clipboard()

In [ ]:
# Export the dataframe
dept_order_agg.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'dept_order_agg.csv'))

**2. Age wise usage Aggregation**

In [ ]:
age_usage_agg = df_1.groupby('age_group').agg({'days_since_prior_order':['mean', 'min','max']})

In [ ]:
age_usage_agg

In [ ]:
# Export the dataframe
age_usage_agg.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'age_usage_agg.csv'))

**3. Age wise purchase Aggregation**

In [ ]:
age_purchase_agg = df_1.groupby('age_group').agg({'prices':['mean', 'min','max']})

In [ ]:
age_purchase_agg

In [ ]:
# Export the dataframe
age_purchase_agg.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'age_purchase_agg.csv'))

**4. Income wise Usage Frequency**

In [ ]:
income_usage_agg = df_1.groupby('Income_Category').agg({'days_since_prior_order':['mean', 'min','max']})

In [ ]:
income_usage_agg

In [ ]:
# Export the dataframe
income_usage_agg.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'income_usage_agg.csv'))

**5. Household Status Usage Frequency**

In [ ]:
hhld_usage_agg = df_1.groupby('Household_Status').agg({'days_since_prior_order':['mean', 'min','max']})

In [ ]:
hhld_usage_agg

In [ ]:
# Export the dataframe
hhld_usage_agg.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'hhld_usage_agg.csv'))

**6. Household Status Spending Aggregation**

In [ ]:
hhld_expediture_agg = df_1.groupby('Household_Status').agg({'prices':['mean', 'min','max']})

In [ ]:
hhld_expediture_agg

In [ ]:
# Export the dataframe
hhld_expediture_agg .to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'hhld_expediture_agg.csv'))

In [ ]:
# Export the final dataset used for visualizations
df_1.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'Instacart_data22.pkl'))

**7. Departments and Price tags**

In [ ]:
# Create a crosstab between household status and day of the week
product_price = pd.crosstab(df_1['department_id'],df_1['prices_range_loc'])

In [ ]:
# Copy the crosstab to excel
product_price